In [92]:
import csv

In [93]:
prop = []
nodes = []
elements = []
condU = []
condF  = []

variablesArray =[
    prop, #P_id;dens;E;v;A
    nodes,#N_id;x;y;;
    elements,#E_id;from;to;prop;
    condU,#I_U_id;N_id;u_x;u_y;
    condF#I_F_id;N_id;Fx;Fy;
]

In [94]:
def dataRowCounter(data,name,dataTo):
    for i in range(len(data)):
        for j in range(len(name)):
            if data[i][0] == name[j]:
                dataTo[j] = i + 2
    dataTo[5] = len(data) + 2

In [95]:
def rangeMaker(ranges,rows): 
    for i in range(len(ranges)):
        ranges[i].append(rows[i])
        ranges[i].append(rows[i+1]-2)

In [96]:
def dataToArray(variab,rang,data):
    for i in range(len(variab)):
        h1 = rang[i][0]
        h2 = rang[i][1]
        for j in range(h1,h2):
            variab[i].append(data[j])

In [97]:
def arrayCleaner(array):
    for data in array:
        for row in data:
            h1 = []
            for i in range(len(row)):
                if row[i] == '':
                    h1.append(i)
            h1.reverse()
            for j in h1:
                del row[j]

In [98]:
def dataPrinter(data):
    for i in data:
        print(i)

In [99]:
def csv_toVar(csv_File,varArray):
    with open(csv_File, "r") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter =";")
        lines = [line for line in csv_reader]
    varNames=[
        "Prop",
        "Nodes",
        "Elements",
        "In_Conds_U",
        "In_Conds_F",
        "End"
    ]
    rowCounter = [
        0,#Prop
        0,#Nodes
        0,#Elements
        0,#In_Conds_U
        0,#In_Conds_F
        0,#End
    ]
    rangeCounter = [
        [],#PropRange:
        [],#NodesRange
        [],#ElementsRange
        [],#In_Conds_URange
        []#In_Conds_FRange
    ]
    dataRowCounter(lines,varNames,rowCounter)
    rangeMaker(rangeCounter,rowCounter)
    dataToArray(varArray,rangeCounter,lines)
    arrayCleaner(varArray)
    dataPrinter(varArray)

In [100]:
csv_toVar('Test.csv',variablesArray)

[['0', '7800', '200000', '0.3', '100']]
[['0', '0', '0'], ['1', '1000', '1000'], ['2', '2000', '1000']]
[['0', '0', '1', '0'], ['1', '1', '2', '0']]
[['0', '0', '1', '1']]
[['0', '2', '0', '10000']]


In [101]:
prop = []
nodes = []
elements = []
condU = []
condF  = []

variablesArray =[
    prop, #P_id;dens;E;v;A
    nodes,#N_id;x;y;;
    elements,#E_id;from;to;prop;
    condU,#I_U_id;N_id;u_x;u_y;
    condF#I_F_id;N_id;Fx;Fy;
]
csv_toVar('Test.csv',variablesArray)

[['0', '7800', '200000', '0.3', '100']]
[['0', '0', '0'], ['1', '1000', '1000'], ['2', '2000', '1000']]
[['0', '0', '1', '0'], ['1', '1', '2', '0']]
[['0', '0', '1', '1']]
[['0', '2', '0', '10000']]


In [102]:
import math
import numpy as np

In [103]:
class Element:
    def __init__(self,id_E,propert,node,element):
        id_E = int(id_E)
        elementRow = 0
        self.x = [0,0] 
        self.y = [0,0]

        for row in element:
          if int(row[0]) == id_E:
              elementRow = row

        self.nodeNum =[
         int(elementRow[1])*2,
         int(elementRow[1])*2+1,
         int(elementRow[2])*2,
         int(elementRow[2])*2+1
        ]
            

        for row in propert:
            if int(elementRow[3]) == int(row[0]):      
                self.dens = float(row[1])
                self.E = float(row[2])
                self.v = float(row[3])
                self.A = 1 #TODO implementation of A into csv
                #self.A = float(row[4])

        for row in node:
            if int(elementRow[1]) == int(row[0]):
                self.x[0] =float(row[1])
                self.y[0] =float(row[2])

        for row in node:
            if int(elementRow[2]) == int(row[0]):
                self.x[1] =float(row[1])
                self.y[1] =float(row[2])        

        self.L = math.sqrt((self.x[0]-self.x[1])**2+(self.y[0]-self.y[1])**2)

        if self.y[1]-self.y[0] == 0:
            if self.x[1]-self.x[0] > 0:
                self.alfa = 0*math.pi/180
            elif self.x[1]-self.x[0] < 0:
                self.alfa = 180*math.pi/180
        elif self.x[1]-self.x[0] == 0:
            if self.y[1]-self.y[0] > 0:
                self.alfa = 90*math.pi/180
            elif self.y[1]-self.y[0] < 0:
                self.alfa = 270*math.pi/180
        else:
            self.alfa = math.atan(self.y[1]-self.y[0])/(self.x[1]-self.x[0]) 

        self.kElementMartix = np.array([
            [math.cos(self.alfa)**2,math.cos(self.alfa)*math.cos(self.alfa),-math.cos(self.alfa)**2,-math.cos(self.alfa)*math.cos(self.alfa)],
            [math.cos(self.alfa)*math.cos(self.alfa),math.sin(self.alfa)**2,-math.cos(self.alfa)*math.cos(self.alfa),-math.sin(self.alfa)**2],
            [-math.cos(self.alfa)**2,-math.cos(self.alfa)*math.cos(self.alfa),math.cos(self.alfa)**2,math.cos(self.alfa)*math.cos(self.alfa)],
            [-math.cos(self.alfa)*math.cos(self.alfa),-math.sin(self.alfa)**2,math.cos(self.alfa)*math.cos(self.alfa),math.sin(self.alfa)**2]
            ])
        self.kElementMartix *=  (self.A*self.E/self.L)

    def printEr(self):
        print(self.x)
        print(self.y)
        print(self.dens)
        print(self.E)
        print(self.v)
        print(self.A)
        print(self.L)
        print(self.alfa)
        print(self.kElementMartix)
        print(self.nodeNum)

In [104]:
e0 =  Element(0,prop,nodes,elements)
e0.printEr()

[0.0, 1000.0]
[0.0, 1000.0]
7800.0
200000.0
0.3
1
1414.213562373095
0.0015697963271282297
[[ 1.41421008e+02  1.41421008e+02 -1.41421008e+02 -1.41421008e+02]
 [ 1.41421008e+02  3.48498777e-04 -1.41421008e+02 -3.48498777e-04]
 [-1.41421008e+02 -1.41421008e+02  1.41421008e+02  1.41421008e+02]
 [-1.41421008e+02 -3.48498777e-04  1.41421008e+02  3.48498777e-04]]
[0, 1, 2, 3]


In [106]:
def elementsMaker(array,eContainer):
    for e in array[2]:
        eContainer.append(Element(e[0],array[0],array[1],array[2]))
    for x in eContainer:
        print(x.kElementMartix)

In [107]:
elementsContainer = []
elementsMaker(variablesArray,elementsContainer)

[[ 1.41421008e+02  1.41421008e+02 -1.41421008e+02 -1.41421008e+02]
 [ 1.41421008e+02  3.48498777e-04 -1.41421008e+02 -3.48498777e-04]
 [-1.41421008e+02 -1.41421008e+02  1.41421008e+02  1.41421008e+02]
 [-1.41421008e+02 -3.48498777e-04  1.41421008e+02  3.48498777e-04]]
[[ 200.  200. -200. -200.]
 [ 200.    0. -200.   -0.]
 [-200. -200.  200.  200.]
 [-200.   -0.  200.    0.]]


In [108]:
def globalKMxMaker(kMx,elem):
    print(len(elem[0].kElementMartix))
    for e in elem:
        for i in range(len(e.kElementMartix)):
            for j in range(len(e.kElementMartix)):
                #print(e.kElementMartix[i][j])
                n1 = e.nodeNum[i]
                n2 = e.nodeNum[j]
                #print(n1,n2)
                kMx[n1][n2] += e.kElementMartix[i][j]
                #print(kMx[n1][n2])


In [109]:
def fivePrec(array):
    for i in range(len(array)):
        for j in range(len(array)):
            array[i][j] = format(array[i][j], ".5g")

In [110]:
mxSize = len(variablesArray[1])*2
globalKMx = np.zeros((mxSize,mxSize))
print(globalKMx)
globalKMxMaker(globalKMx,elementsContainer)
fivePrec(globalKMx)
print(globalKMx)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
4
[[ 1.4142e+02  1.4142e+02 -1.4142e+02 -1.4142e+02  0.0000e+00  0.0000e+00]
 [ 1.4142e+02  3.4850e-04 -1.4142e+02 -3.4850e-04  0.0000e+00  0.0000e+00]
 [-1.4142e+02 -1.4142e+02  3.4142e+02  3.4142e+02 -2.0000e+02 -2.0000e+02]
 [-1.4142e+02 -3.4850e-04  3.4142e+02  3.4850e-04 -2.0000e+02  0.0000e+00]
 [ 0.0000e+00  0.0000e+00 -2.0000e+02 -2.0000e+02  2.0000e+02  2.0000e+02]
 [ 0.0000e+00  0.0000e+00 -2.0000e+02  0.0000e+00  2.0000e+02  0.0000e+00]]


In [111]:
globalRedKMx = np.copy(globalKMx)
print(globalRedKMx)
gr = np.delete(globalRedKMx,[0,1],0) #<-- array, what i, row/coloum
gr2 = np.delete(gr,[0,1],1) #<-- array, what i, row/coloum
def globalRedKMxMaker(kMx,inU):
    hArray = []
    for init in inU:
        print(init)
        if int(init[2]) == 1:
            hArray.append(int(init[1])*2)
        if int(init[3]) == 1:
            hArray.append(int(init[1])*2+1)
    h1 = np.delete(kMx,hArray,0)
    h2 = np.delete(h1,hArray,1)
    return np.copy(h2)

globalRedKMx = globalRedKMxMaker(globalRedKMx,variablesArray[3])
print(globalRedKMx)

[[ 1.4142e+02  1.4142e+02 -1.4142e+02 -1.4142e+02  0.0000e+00  0.0000e+00]
 [ 1.4142e+02  3.4850e-04 -1.4142e+02 -3.4850e-04  0.0000e+00  0.0000e+00]
 [-1.4142e+02 -1.4142e+02  3.4142e+02  3.4142e+02 -2.0000e+02 -2.0000e+02]
 [-1.4142e+02 -3.4850e-04  3.4142e+02  3.4850e-04 -2.0000e+02  0.0000e+00]
 [ 0.0000e+00  0.0000e+00 -2.0000e+02 -2.0000e+02  2.0000e+02  2.0000e+02]
 [ 0.0000e+00  0.0000e+00 -2.0000e+02  0.0000e+00  2.0000e+02  0.0000e+00]]
['0', '0', '1', '1']
[[ 3.4142e+02  3.4142e+02 -2.0000e+02 -2.0000e+02]
 [ 3.4142e+02  3.4850e-04 -2.0000e+02  0.0000e+00]
 [-2.0000e+02 -2.0000e+02  2.0000e+02  2.0000e+02]
 [-2.0000e+02  0.0000e+00  2.0000e+02  0.0000e+00]]


In [112]:
def initFToArrayMaker(inF):
    h1 = np.zeros((mxSize,1))

    for i in inF:
        for j in range(0,2):
            h1[int(i[1])*2+j][0] = int(i[2+j])
    return np.copy(h1)
fMx = initFToArrayMaker(variablesArray[4])
print(fMx)

def fTofRed(inU,array):
    hArray = []
    for init in inU:
        print(init)
        if int(init[2]) == 1:
            hArray.append(int(init[1])*2)
        if int(init[3]) == 1:
            hArray.append(int(init[1])*2+1)
    h1 = np.delete(array,hArray,0)
    return np.copy(h1)

fMxRed = fTofRed(variablesArray[3],fMx)
print(fMxRed) #!!!!!!!!!!!!!BUG talán indexeléssel

[[    0.]
 [    0.]
 [    0.]
 [    0.]
 [    0.]
 [10000.]]
['0', '0', '1', '1']
[[    0.]
 [    0.]
 [    0.]
 [10000.]]


In [113]:
print(fMxRed)
print(globalRedKMx)

[[    0.]
 [    0.]
 [    0.]
 [10000.]]
[[ 3.4142e+02  3.4142e+02 -2.0000e+02 -2.0000e+02]
 [ 3.4142e+02  3.4850e-04 -2.0000e+02  0.0000e+00]
 [-2.0000e+02 -2.0000e+02  2.0000e+02  2.0000e+02]
 [-2.0000e+02  0.0000e+00  2.0000e+02  0.0000e+00]]


In [114]:
try1 = np.linalg.inv(globalRedKMx)
try2 = np.multiply(fMxRed,try1)
try3 = try1*fMxRed
try4 = np.matmul(try1,fMxRed)
try5 = try1@fMxRed
print(try1)
#print(try2)
#print(try3)
#print(try4)
print(try5)

[[-1.74253772e-08  7.07115305e-03 -1.74253772e-08  7.07115305e-03]
 [ 7.07115305e-03 -7.07115305e-03  7.07115305e-03 -7.07115305e-03]
 [-1.74253772e-08  7.07115305e-03 -1.74253772e-08  1.20711530e-02]
 [ 7.07115305e-03 -7.07115305e-03  1.20711530e-02 -1.20711530e-02]]
[[  70.7115305]
 [ -70.7115305]
 [ 120.7115305]
 [-120.7115305]]


In [105]:
import matplotlib.pyplot as plt